In [1]:
import pandas as pd 
import numpy as np 

GainRatio is used to select the node splitting attribute and the splitting point of the continuous attribute value in the C4.5 algorithm

A_i is an attribute in the attribute set A to be split, a_1 and a_2 are weight coefficients assigned to the information gain and the gainratio: 0 <= a_1, a_2 <= 1, and a_1 + a_2 = 1.

IG_GR(Data, A_i) = (a_1 + a_2 / SplitInformation(D, A_i)) * InfoGain(D, A_i)

##### C4.5 
https://github.com/barisesmer/C4.5/tree/master/c45

https://github.com/Valdecy/C4.5/blob/master/Python-DM-Classification-04-C4.5.py

Code of Node Class and Tree and modified from 

https://github.com/loginaway/DecisionTree/blob/master/Node.py

https://github.com/loginaway/DecisionTree/blob/master/DecisionTree.py

In [257]:
import numpy as np

class Node(object):
    '''
    Basic element for decision tree.
    Initialization: to initialize a tree node, at least feature_name is needed.
    Connection: to connect different nodes, there are two situations.
        1, if the parent node is discrete (i.e. node.discrete==True), 
            then 
            >> node[feature_value]=childnode 
            will do the connection.
        2, if the parent node is continuous, then 
            >> node['<=']=childnode 
            or 
            >> node['>']=childnode 
            will assign its children. To be specific, 
            node['<=']=childnode means if the feature value is bigger than 
            node.threshold, then the workflow goes left to the childnode, and vice versa.
    
    Classification: if you have constructed a tree, you may want to classify a 
        certain group of data. Simply call the root node will do this task, i.e.
        >> root(data)
        where data should be a dictionary containing the corresponding key and value.
        And this method will return the classification result.
    Visualization: A naive visualization is implemented here. 
        For example, you may visualize a subtree rooted at 'node' by
        >> print(node)
        The tree structure will then be printed on the console.
    '''
    def __init__(self, feature_name='', discrete=True, threshold=0, isLeaf=False, classification=None):
        # for discrete node: the next node can be retrieved by self.map[feature_value]
        # for continuous node: by self.map['<='] (when value <= self.threshold)
        #                       by self.map['>'] (when value > self.threshold)
        self.map=dict()

        self.discrete=discrete
        self.feature_name=feature_name
        self.isLeaf=isLeaf
        if isLeaf:
            self.classification=classification
        if not discrete:
            self.threshold=threshold

    def __setitem__(self, key, value):
        self.map[key]=value

    def __getitem__(self, key):
        return self.map.get(key)

    def __call__(self, data):
        '''
        This method should be used on the root to predict its classification.
        data: a dict with its key being the features and value being 
        the corresponding value.
        '''
        if self.isLeaf:
            return self.classification
        # if the node has a discrete feature, use __getitem__ to find the next node
        # then call the next node.
        if self.discrete:
            return self.map[data[self.feature_name]](data)
        else:
        # node is not discrete: print the self.feature_name
            #print("continuous name")
            #print(self.feature_name)
            if data[self.feature_name].astype(np.float32)>self.threshold:
                return self.map['>'](data)
            else:
                return self.map['<='](data)

    def __str__(self):
        '''
        This method is designed for printing the subtree rooted at the current node.
        To print the tree, try print(self).
        '''
        hierarchy={}
        stack=[(0, 0, self)]
        count=0
        while stack:
            # BFS
            (layer_index, node_index, current)=stack.pop(0)
            layer_index+=1

            hierarchy[layer_index]=hierarchy.get(layer_index, [])
            hierarchy[layer_index].append((node_index, current.feature_name,\
                    [(str(current.classification),)] if current.isLeaf else \
                     [(str(key), '' if current.discrete else str(current.threshold), item.feature_name) for key, item in current.map.items()]))
            # print(hierarchy)

            if current.isLeaf:
                continue
            for _, item in current.map.items():
                count+=1
                stack.append((layer_index, count, item))
            # print(item_index_map)

        totallist=[]
        for layer, layer_content in hierarchy.items():
            rowlist=[]
            for i in layer_content:
                rowlist.append('['+i[1]+']'+': {'+', '.join([' '.join(item) for item in i[2]])+'}')
            rowstr=' +++ '.join(rowlist)
            totallist.append(rowstr)

        return '============Root===========\n'+'\n\n'.join(totallist)+'\n============Leaf==========='

In [256]:
# OG
#import sys
import numpy as np
#from Node import Node

# set the maximal recursion limits here.
#sys.setrecursionlimit(10000)

class baseClassDecisionTree(object):

    '''
    The main class of decision tree.
    '''
    def __init__(self, feature_discrete=[], treeType='C4.5'):
        '''
        feature_discrete: a dict with its each key-value pair being (feature_name: True/False),
            where True means the feature is discrete and False means the feature is 
            continuous. 
        type: ID3/C4.5/CART
        pruning: pre/post
        '''

        self.feature_discrete=feature_discrete
        self.treeType=treeType
        self.leaf_count=0
        self.tmp_classification=''
        self._root_node = root
        self.tree=None

    def Entropy(self, list_of_class):
        '''
        Compute the entropy for the given list of class.
        list_of_class: an array of classification labels, e.g. ['duck', 'duck', 'dolphin']
        'duck': 2/3, 'dolphin': 1/3, so the entropy for this array is 0.918
        '''
        count={}
        for key in list_of_class:
            count[key]=count.get(key, 0)+1
        frequency=np.array(tuple(count.values()))/len(list_of_class)
        return -1*np.vdot(frequency, np.log2(frequency))

    def Information_Gain(self, list_of_class, grouped_list_of_class):
        '''
        Compute the Information Gain.
        list_of_class: an array of classification labels, e.g. ['duck', 'duck', 'dolphin']
        grouped_list_of_class: the list of class grouped by the values of 
            a certain attribute, e.g. [('duck'), ('duck', 'dolphin')].
        The Information_Gain for this example is 0.2516.
        '''
        sec2=np.sum([len(item)*self.Entropy(item) for item in grouped_list_of_class])/len(list_of_class)
        return self.Entropy(list_of_class)-sec2


    def Information_Ratio(self, list_of_class, grouped_list_of_class):
        '''
        Compute the Information Ratio.
        list_of_class: an array of classification labels, e.g. ['duck', 'duck', 'dolphin']
        grouped_list_of_class: the list of class grouped by the values of 
            a certain attribute, e.g. [('duck'), ('duck', 'dolphin')].
        The Information_Ratio for this example is 0.2740.
        '''
        tmp=np.array([len(item)/len(list_of_class) for item in grouped_list_of_class])
        
        # Here we assume instrinsic_value is SplitInformation! 
        intrinsic_value=-1*np.vdot(tmp, np.log2(tmp))

        return self.Information_Gain(list_of_class, grouped_list_of_class) / intrinsic_value

    def IGGR(self, list_of_class, grouped_list_of_class, a1, a2):

        sec2 = np.sum([len(item)*self.Entropy(item) for item in grouped_list_of_class])/len(list_of_class)
        
        infoGain =  self.Entropy(list_of_class)-sec2

        #print("IGGR grouped_list_of_class")
        #print("----------")
        #print(grouped_list_of_class)
        #print("----------")

        tmp = np.array([len(item)/len(list_of_class) for item in grouped_list_of_class])
        
        intrinsic_value = -1*np.vdot(tmp, np.log2(tmp))

        GainRatio = self.Information_Gain(list_of_class, grouped_list_of_class)/ intrinsic_value

        sol1 = a1 * infoGain + a2 * GainRatio
        sol2 = (a1 + ( a2 / intrinsic_value) ) * infoGain 

        return sol2

    def orderByGainOrRatio(self, D, A, by='Gain'):
        '''
        Return the order by Information Gain or Information Ratio.
        by: 'Gain', 'Ratio'.
        For the definition of D and A, see the remark in method 'fit'.
        '''
        tmp_value_dict=dict()

        target_function = self.Information_Gain if by=='Gain' else self.Information_Ratio

        for attr, info in A.items():
            possibleVal=np.unique(D[:, info[0]])
            # if the continuous attribute have only one possible value, then 
            # choosing it won't improve the model, so we abandon it.
            if len(possibleVal)==1:
                continue

            if self.feature_discrete[attr] is True:
                # discrete
                if len(info)<2:
                    A[attr].append(possibleVal)
                # retrieve the grouped list of class
                grouped_list_of_class=[]

                for val in possibleVal:

                    indexes=np.argwhere(D[:, info[0]]==val)
                    grouped_list_of_class.append(D[indexes, -1].flatten())

                IC_value=target_function(D[:, -1], grouped_list_of_class)
                tmp_value_dict[attr]=IC_value

            else:

                # continuous
                split_points=(possibleVal[: -1].astype(np.float32)+possibleVal[1:].astype(np.float32))/2
                maxMetric=-1

                for point in split_points:
                    smaller_set=D[np.argwhere(D[:, info[0]]<=str(point)), -1].flatten()
                    bigger_set=D[np.argwhere(D[:, info[0]]>str(point)), -1].flatten()

                    # compute the metric
                    IC_tmp=target_function(D[:, -1], (smaller_set, bigger_set))
                    if IC_tmp>maxMetric:
                        maxMetric=IC_tmp
                        threshold=point

                # set the threshold
                if len(info)<2:
                    A[attr].append(threshold)
                else:
                    A[attr][1]=threshold
                tmp_value_dict[attr]=maxMetric

        # find the attribute with the max tmp_value_dict value
        attr_list=list(tmp_value_dict.keys())
        attr_list.sort(key=lambda x: tmp_value_dict[x])

        return attr_list

    def orderByIGGR(self, D, A, a1, a2):
        '''
        Return the order by Information Gain or Information Ratio.
        by: 'Gain', 'Ratio'.
        For the definition of D and A, see the remark in method 'fit'.
        '''
        tmp_value_dict=dict()

        target_function1 = self.Information_Gain
        target_function2 = self.Information_Ratio
        target_function3 = self.IGGR
        
        for attr, info in A.items():
            possibleVal=np.unique(D[:, info[0]])
            # if the continuous attribute have only one possible value, then 
            # choosing it won't improve the model, so we abandon it.
            if len(possibleVal)==1:
                continue

            if self.feature_discrete[attr] is True:
                # discrete
                if len(info)<2:
                    A[attr].append(possibleVal)
                # retrieve the grouped list of class
                grouped_list_of_class=[]
                for val in possibleVal:
                    indexes=np.argwhere(D[:, info[0]]==val)
                    grouped_list_of_class.append(D[indexes, -1].flatten())
                IC_value=target_function3(D[:, -1], grouped_list_of_class, a1, a2)
                tmp_value_dict[attr]=IC_value

            else:
                # continuous
                split_points=(possibleVal[: -1].astype(np.float32)+possibleVal[1:].astype(np.float32))/2
                maxMetric=-1
                for point in split_points:
                    smaller_set=D[np.argwhere(D[:, info[0]]<=str(point)), -1].flatten()
                    bigger_set=D[np.argwhere(D[:, info[0]]>str(point)), -1].flatten()

                    # compute the metric
                    # list_of_class, grouped_list_of_class, a1, a2
                    IC_tmp=target_function3(D[:, -1], (smaller_set, bigger_set), a1, a2)
                    if IC_tmp>maxMetric:
                        maxMetric=IC_tmp
                        threshold=point

                # set the threshold
                if len(info)<2:
                    A[attr].append(threshold)
                else:
                    A[attr][1]=threshold
                tmp_value_dict[attr]=maxMetric

        # find the attribute with the max tmp_value_dict value
        attr_list=list(tmp_value_dict.keys())
        attr_list.sort(key=lambda x: tmp_value_dict[x])

        return attr_list

    def chooseAttribute(self, D, A):

        '''
        Choose an attribute from A according to the metrics above.
        For the definition of D and A, see method 'fit'.
        Different principal for different tree types:
        ID3: choose the attribute that maximizes the Information Gain.
        C4.5: 
            1, choose those attributes whose Information Gain are above average.
            2, choose the one that maximizes the Gain Ratio from these attributes.
        CART: choose the attribute that minimizes the Gini Index.
        IG_GR: 
        '''
        if self.treeType=='ID3':
            attr_list=self.orderByGainOrRatio(D, A, by='Gain')
            return attr_list[-1]

        if self.treeType=='C4.5':
            attr_list=self.orderByGainOrRatio(D, A, by='Gain')

            # for C4.5, we choose the attributes whose Gain are above average
            # and then order them by Ratio.

            sub_A={key: A[key] for key in attr_list}
            attr_list=self.orderByGainOrRatio(D, sub_A, by='Ratio')

            return attr_list[-1]

        if self.treeType=='IGGR':

            attr_list=self.orderByIGGR(D, A, 0.3, 0.7)

            return attr_list[-1]

    def fit(self, D, A):
        '''
        Train the tree.
        To save the training result:
        >> self.tree=self.fit(D, A)
        D: the training set, a size [m, n+1] numpy array (with str type elements), 
            where m is the number of training data and n is the number of attributes.
            The last column of D is the classifications (or labels).
        A: the attributes set. It is a dict with its structure being like 
            {attr_name: [index_in_D_columns, possibleVal_or_threshold], ...}
            attr_name: name of the attribute
            index_in_D_columns: the corresponding index of the attribute in ndarray D (starting from 0)
            possibleVal_or_threshold: 
                ###################################################
                ## This value may not always be available in A   ##
                ## it is added after 'chooseAttribute' is called ##
                ## And it will be updated after each call        ##
                ###################################################
                1, if the attribute is discrete, then it is a ndarray containing all possible values 
                    of this attribute.
                2, if the attribute is continuous, then possibleVal_or_threshold is the most recent 
                    threshold.
        '''
        if len(D)==0:
            node=Node(feature_name='leaf-'+str(self.leaf_count), isLeaf=True, \
                classification=self.tmp_classification)
            self.leaf_count+=1
            return node

        if len(np.unique(D[:, -1]))<=1:
            node=Node(feature_name='leaf-'+str(self.leaf_count), isLeaf=True, \
                classification=D[0, -1])
            self.leaf_count+=1
            return node

        if len(A)==0 or len(np.unique(D[:, :-1], axis=0))<=1:
            count_dict={}
            for key in D[:, -1]:
                count_dict[key]=count_dict.get(key, 0)+1
            most_frequent=sorted(D[:, -1], key=lambda x: count_dict[x])[-1]

            node=Node(feature_name='leaf-'+str(self.leaf_count), isLeaf=True, \
                classification=most_frequent)
            self.leaf_count+=1
            return node 

        count_dict={}
        for key in D[:, -1]:
            count_dict[key]=count_dict.get(key, 0)+1
        most_frequent=sorted(D[:, -1], key=lambda x: count_dict[x])[-1]
        self.tmp_classification=most_frequent

        # choose target attribute
        target_attr=self.chooseAttribute(D, A)
        # print(target_attr)

        # generate nodes for each possible values of the target attribute if it's discrete
        # generate two nodes for the two classification if it's continuous
        # related information is stored in A[target_attr][1] now, 
        # since we have called chooseAttribute at least once.
        info=A[target_attr]

        if self.feature_discrete[target_attr]:

            node=Node(feature_name=target_attr, discrete=True, isLeaf=False)
            # generate nodes for each possible values
            for possibleVal in info[1]:
                keys=set(A.keys()).difference({target_attr})
                # connect node to its child
                tmp_D=D[np.argwhere(D[:, info[0]]==possibleVal), :]
                tmp_A={key: A[key] for key in keys}

                node[possibleVal]=self.fit(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), tmp_A)
        
        else:
            # continuous
            threshold=info[1]
            node=Node(feature_name=target_attr, discrete=False, threshold=threshold,isLeaf=False)

            tmp_D=D[np.argwhere(D[:, info[0]]<=str(threshold)), :]
            node['<=']=self.fit(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), A)

            tmp_D=D[np.argwhere(D[:, info[0]]>str(threshold)), :]
            node['>']=self.fit(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), A)
            
        
        return node

    def post_prune(self, training_D, testing_D, A, current=None, parent=None):
        '''
        self.tree is required.
        This method conducts the post-pruning to enhance the model performance.
        To make sure this method will work, set 
        >> current=self.tree
        when you call it.
        '''
        self.current_accuracy=self.evaluate(testing_D, A)
        count_dict={}
        if len(training_D)==0:
            return 
        # print(training_D)
        for key in training_D[:, -1]:
            count_dict[key]=count_dict.get(key, 0)+1
        most_frequent=sorted(training_D[:, -1], key=lambda x: count_dict[x])[-1]

        leaf_parent=True
        for key, node in current.map.items():
            if not node.isLeaf:
                leaf_parent=False
                # Recursion, DFS
                if node.discrete:
                    tmp_D=training_D[np.argwhere(training_D[:, A[current.feature_name][0]]==key), :]
                else:
                    if key=='<=':
                        tmp_D=training_D[np.argwhere(training_D[:, A[current.feature_name][0]]<=str(node.threshold)), :]
                    else:
                        tmp_D=training_D[np.argwhere(training_D[:, A[current.feature_name][0]]>str(node.threshold)), :]
                self.post_prune(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), testing_D, A, parent=current, current=node)
        
        tmp_node=Node(feature_name='leaf-'+str(self.leaf_count), isLeaf=True, classification=most_frequent)
        
        if parent:
            # when current node is not the root
            for key, node in parent.map.items():
                if node==current:
                    parent.map[key]=tmp_node
                    saved_key=key
                    break
            # compare the evaluation, if it is enhanced then prune the tree.
            tmp_accuracy=self.evaluate(testing_D, A)
            if tmp_accuracy<self.current_accuracy:
                parent.map[saved_key]=current
            else:
                self.current_accuracy=tmp_accuracy
                self.leaf_count+=1
            return
        else:
            # when current node is the root
            saved_tree=self.tree
            self.tree=tmp_node
            tmp_accuracy=self.evaluate(testing_D, A)
            if tmp_accuracy<self.current_accuracy:
                self.tree=saved_tree
            else:
                self.current_accuracy=tmp_accuracy
                self.leaf_count+=1
            return

    def predict(self, D, A):
        '''
        Predict the classification for the data in D.
        For the definition of A, see method 'fit'. 
        There is one critical difference between D and that defined in 'fit':
            the last column may or may not be the labels. 
            This method works as long as the feature index in A matches the corresponding
            column in D.
        '''
        row, _=D.shape
        pred=np.empty((row, 1), dtype=str)
        tmp_data={key: None for key in A.keys()}
        for i in range(len(D)):
            for key, info in A.items():
                tmp_data[key]=D[i, info[0]]
            pred[i]=self.tree(tmp_data)
        return pred
    
    def evaluate(self, testing_D, A):
        '''
        Evaluate the performance of decision tree. (Accuracy)
        For definition of testing_D and A, see 'predict'.
        However, here the testing_D is required to be labelled, that is, its last column 
        should be labels of the data.
        '''
        true_label=testing_D[:, -1]
        pred_label=self.predict(testing_D, A)
        
        success_count=0
        for i in range(len(true_label)):
            if true_label[i]==pred_label[i]:
                success_count+=1

        return success_count/len(true_label)

        

class Tree_DPDT(baseClassDecisionTree):
    def __init__(self, conf):

        super().__init__(conf['feature_discrete'], conf['treeType'])
        self.conf=conf

    # try rename 'train' as 'fit' and run again, what is wrong?
    def train(self, D):
        self.tree=super().fit(D, self.conf['A'])
        
    def prune(self, training_D, testing_D):
        super().post_prune(training_D, testing_D, self.conf['A'], current=self.tree)

    def pred(self, D):
        return super().predict(D, self.conf['A'])
    
    def eval(self, D):
        return super().evaluate(D, self.conf['A'])

In [ ]:
#import sys
import numpy as np
#from Node import Node

# set the maximal recursion limits here.
#sys.setrecursionlimit(10000)

class Tree_DPDT(baseClassDecisionTree):
    def __init__(self, conf):

        super().__init__(conf['feature_discrete'], conf['treeType'])
        self.conf=conf

    # try rename 'train' as 'fit' and run again, what is wrong?
    def train(self, D):
        self.tree=super().fit(D, self.conf['A'])
        
    def prune(self, training_D, testing_D):
        super().post_prune(training_D, testing_D, self.conf['A'], current=self.tree)

    def pred(self, D):
        return super().predict(D, self.conf['A'])
    
    def eval(self, D):
        return super().evaluate(D, self.conf['A'])

class baseClassDecisionTree(object):

    '''
    The main class of decision tree.
    '''
    def __init__(self, feature_discrete=[], treeType='C4.5'):
        '''
        feature_discrete: a dict with its each key-value pair being (feature_name: True/False),
            where True means the feature is discrete and False means the feature is 
            continuous. 
        type: ID3/C4.5/CART
        pruning: pre/post
        '''

        self.feature_discrete=feature_discrete
        self.treeType=treeType
        self.leaf_count=0
        self.tmp_classification=''
        self._root_node = root
        self.tree=None

    def Entropy(self, list_of_class):
        '''
        Compute the entropy for the given list of class.
        list_of_class: an array of classification labels, e.g. ['duck', 'duck', 'dolphin']
        'duck': 2/3, 'dolphin': 1/3, so the entropy for this array is 0.918
        '''
        count={}
        for key in list_of_class:
            count[key]=count.get(key, 0)+1
        frequency=np.array(tuple(count.values()))/len(list_of_class)
        return -1*np.vdot(frequency, np.log2(frequency))

    def Information_Gain(self, list_of_class, grouped_list_of_class):
        '''
        Compute the Information Gain.
        list_of_class: an array of classification labels, e.g. ['duck', 'duck', 'dolphin']
        grouped_list_of_class: the list of class grouped by the values of 
            a certain attribute, e.g. [('duck'), ('duck', 'dolphin')].
        The Information_Gain for this example is 0.2516.
        '''
        sec2=np.sum([len(item)*self.Entropy(item) for item in grouped_list_of_class])/len(list_of_class)
        return self.Entropy(list_of_class)-sec2


    def Information_Ratio(self, list_of_class, grouped_list_of_class):
        '''
        Compute the Information Ratio.
        list_of_class: an array of classification labels, e.g. ['duck', 'duck', 'dolphin']
        grouped_list_of_class: the list of class grouped by the values of 
            a certain attribute, e.g. [('duck'), ('duck', 'dolphin')].
        The Information_Ratio for this example is 0.2740.
        '''
        tmp=np.array([len(item)/len(list_of_class) for item in grouped_list_of_class])
        
        # Here we assume instrinsic_value is SplitInformation! 
        intrinsic_value=-1*np.vdot(tmp, np.log2(tmp))

        return self.Information_Gain(list_of_class, grouped_list_of_class) / intrinsic_value

    def IGGR(self, list_of_class, grouped_list_of_class, a1, a2):

        sec2 = np.sum([len(item)*self.Entropy(item) for item in grouped_list_of_class])/len(list_of_class)
        
        infoGain =  self.Entropy(list_of_class)-sec2

        #print("IGGR grouped_list_of_class")
        #print("----------")
        #print(grouped_list_of_class)
        #print("----------")

        tmp = np.array([len(item)/len(list_of_class) for item in grouped_list_of_class])
        
        intrinsic_value = -1*np.vdot(tmp, np.log2(tmp))

        GainRatio = self.Information_Gain(list_of_class, grouped_list_of_class)/ intrinsic_value

        sol1 = a1 * infoGain + a2 * GainRatio
        sol2 = (a1 + ( a2 / intrinsic_value) ) * infoGain 

        return sol2

    def orderByGainOrRatio(self, D, A, by='Gain'):
        '''
        Return the order by Information Gain or Information Ratio.
        by: 'Gain', 'Ratio'.
        For the definition of D and A, see the remark in method 'fit'.
        '''
        tmp_value_dict=dict()

        target_function = self.Information_Gain if by=='Gain' else self.Information_Ratio

        for attr, info in A.items():
            possibleVal=np.unique(D[:, info[0]])
            # if the continuous attribute have only one possible value, then 
            # choosing it won't improve the model, so we abandon it.
            if len(possibleVal)==1:
                continue

            if self.feature_discrete[attr] is True:
                # discrete
                if len(info)<2:
                    A[attr].append(possibleVal)
                # retrieve the grouped list of class
                grouped_list_of_class=[]

                for val in possibleVal:

                    indexes=np.argwhere(D[:, info[0]]==val)
                    grouped_list_of_class.append(D[indexes, -1].flatten())

                IC_value=target_function(D[:, -1], grouped_list_of_class)
                tmp_value_dict[attr]=IC_value

            else:

                # continuous
                split_points=(possibleVal[: -1].astype(np.float32)+possibleVal[1:].astype(np.float32))/2
                maxMetric=-1

                for point in split_points:
                    smaller_set=D[np.argwhere(D[:, info[0]]<=str(point)), -1].flatten()
                    bigger_set=D[np.argwhere(D[:, info[0]]>str(point)), -1].flatten()

                    # compute the metric
                    IC_tmp=target_function(D[:, -1], (smaller_set, bigger_set))
                    if IC_tmp>maxMetric:
                        maxMetric=IC_tmp
                        threshold=point

                # set the threshold
                if len(info)<2:
                    A[attr].append(threshold)
                else:
                    A[attr][1]=threshold
                tmp_value_dict[attr]=maxMetric

        # find the attribute with the max tmp_value_dict value
        attr_list=list(tmp_value_dict.keys())
        attr_list.sort(key=lambda x: tmp_value_dict[x])

        return attr_list

    def orderByIGGR(self, D, A, a1, a2):
        '''
        Return the order by Information Gain or Information Ratio.
        by: 'Gain', 'Ratio'.
        For the definition of D and A, see the remark in method 'fit'.
        '''
        tmp_value_dict=dict()

        target_function1 = self.Information_Gain
        target_function2 = self.Information_Ratio
        target_function3 = self.IGGR
        
        for attr, info in A.items():
            possibleVal=np.unique(D[:, info[0]])
            # if the continuous attribute have only one possible value, then 
            # choosing it won't improve the model, so we abandon it.
            if len(possibleVal)==1:
                continue

            if self.feature_discrete[attr] is True:
                # discrete
                if len(info)<2:
                    A[attr].append(possibleVal)
                # retrieve the grouped list of class
                grouped_list_of_class=[]
                for val in possibleVal:
                    indexes=np.argwhere(D[:, info[0]]==val)
                    grouped_list_of_class.append(D[indexes, -1].flatten())
                IC_value=target_function3(D[:, -1], grouped_list_of_class, a1, a2)
                tmp_value_dict[attr]=IC_value

            else:
                # continuous
                split_points=(possibleVal[: -1].astype(np.float32)+possibleVal[1:].astype(np.float32))/2
                maxMetric=-1
                for point in split_points:
                    smaller_set=D[np.argwhere(D[:, info[0]]<=str(point)), -1].flatten()
                    bigger_set=D[np.argwhere(D[:, info[0]]>str(point)), -1].flatten()

                    # compute the metric
                    # list_of_class, grouped_list_of_class, a1, a2
                    IC_tmp=target_function3(D[:, -1], (smaller_set, bigger_set), a1, a2)
                    if IC_tmp>maxMetric:
                        maxMetric=IC_tmp
                        threshold=point

                # set the threshold
                if len(info)<2:
                    A[attr].append(threshold)
                else:
                    A[attr][1]=threshold
                tmp_value_dict[attr]=maxMetric

        # find the attribute with the max tmp_value_dict value
        attr_list=list(tmp_value_dict.keys())
        attr_list.sort(key=lambda x: tmp_value_dict[x])

        return attr_list

    def chooseAttribute(self, D, A):

        '''
        Choose an attribute from A according to the metrics above.
        For the definition of D and A, see method 'fit'.
        Different principal for different tree types:
        ID3: choose the attribute that maximizes the Information Gain.
        C4.5: 
            1, choose those attributes whose Information Gain are above average.
            2, choose the one that maximizes the Gain Ratio from these attributes.
        CART: choose the attribute that minimizes the Gini Index.
        IG_GR: 
        '''
        if self.treeType=='ID3':
            attr_list=self.orderByGainOrRatio(D, A, by='Gain')
            return attr_list[-1]

        if self.treeType=='C4.5':
            attr_list=self.orderByGainOrRatio(D, A, by='Gain')

            # for C4.5, we choose the attributes whose Gain are above average
            # and then order them by Ratio.

            sub_A={key: A[key] for key in attr_list}
            attr_list=self.orderByGainOrRatio(D, sub_A, by='Ratio')

            return attr_list[-1]

        if self.treeType=='IGGR':

            attr_list=self.orderByIGGR(D, A, 0.3, 0.7)

            return attr_list[-1]

    def fit(self, D, A):
        '''
        Train the tree.
        To save the training result:
        >> self.tree=self.fit(D, A)
        D: the training set, a size [m, n+1] numpy array (with str type elements), 
            where m is the number of training data and n is the number of attributes.
            The last column of D is the classifications (or labels).
        A: the attributes set. It is a dict with its structure being like 
            {attr_name: [index_in_D_columns, possibleVal_or_threshold], ...}
            attr_name: name of the attribute
            index_in_D_columns: the corresponding index of the attribute in ndarray D (starting from 0)
            possibleVal_or_threshold: 
                ###################################################
                ## This value may not always be available in A   ##
                ## it is added after 'chooseAttribute' is called ##
                ## And it will be updated after each call        ##
                ###################################################
                1, if the attribute is discrete, then it is a ndarray containing all possible values 
                    of this attribute.
                2, if the attribute is continuous, then possibleVal_or_threshold is the most recent 
                    threshold.
        '''
        if len(D)==0:
            node=Node(feature_name='leaf-'+str(self.leaf_count), isLeaf=True, \
                classification=self.tmp_classification)
            self.leaf_count+=1
            return node

        if len(np.unique(D[:, -1]))<=1:
            node=Node(feature_name='leaf-'+str(self.leaf_count), isLeaf=True, \
                classification=D[0, -1])
            self.leaf_count+=1
            return node

        if len(A)==0 or len(np.unique(D[:, :-1], axis=0))<=1:
            count_dict={}
            for key in D[:, -1]:
                count_dict[key]=count_dict.get(key, 0)+1
            most_frequent=sorted(D[:, -1], key=lambda x: count_dict[x])[-1]

            node=Node(feature_name='leaf-'+str(self.leaf_count), isLeaf=True, \
                classification=most_frequent)
            self.leaf_count+=1
            return node 

        count_dict={}
        for key in D[:, -1]:
            count_dict[key]=count_dict.get(key, 0)+1
        most_frequent=sorted(D[:, -1], key=lambda x: count_dict[x])[-1]
        self.tmp_classification=most_frequent

        # choose target attribute
        target_attr=self.chooseAttribute(D, A)
        # print(target_attr)

        # generate nodes for each possible values of the target attribute if it's discrete
        # generate two nodes for the two classification if it's continuous
        # related information is stored in A[target_attr][1] now, 
        # since we have called chooseAttribute at least once.
        info=A[target_attr]

        if self.feature_discrete[target_attr]:

            node=Node(feature_name=target_attr, discrete=True, isLeaf=False)
            # generate nodes for each possible values
            for possibleVal in info[1]:
                keys=set(A.keys()).difference({target_attr})
                # connect node to its child
                tmp_D=D[np.argwhere(D[:, info[0]]==possibleVal), :]
                tmp_A={key: A[key] for key in keys}

                node[possibleVal]=self.fit(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), tmp_A)
        
        else:
            # continuous
            threshold=info[1]
            node=Node(feature_name=target_attr, discrete=False, threshold=threshold,isLeaf=False)

            tmp_D=D[np.argwhere(D[:, info[0]]<=str(threshold)), :]
            node['<=']=self.fit(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), A)

            tmp_D=D[np.argwhere(D[:, info[0]]>str(threshold)), :]
            node['>']=self.fit(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), A)
            
        
        return node

    def post_prune(self, training_D, testing_D, A, current=None, parent=None):
        '''
        self.tree is required.
        This method conducts the post-pruning to enhance the model performance.
        To make sure this method will work, set 
        >> current=self.tree
        when you call it.
        '''
        self.current_accuracy=self.evaluate(testing_D, A)
        count_dict={}
        if len(training_D)==0:
            return 
        # print(training_D)
        for key in training_D[:, -1]:
            count_dict[key]=count_dict.get(key, 0)+1
        most_frequent=sorted(training_D[:, -1], key=lambda x: count_dict[x])[-1]

        leaf_parent=True
        for key, node in current.map.items():
            if not node.isLeaf:
                leaf_parent=False
                # Recursion, DFS
                if node.discrete:
                    tmp_D=training_D[np.argwhere(training_D[:, A[current.feature_name][0]]==key), :]
                else:
                    if key=='<=':
                        tmp_D=training_D[np.argwhere(training_D[:, A[current.feature_name][0]]<=str(node.threshold)), :]
                    else:
                        tmp_D=training_D[np.argwhere(training_D[:, A[current.feature_name][0]]>str(node.threshold)), :]
                self.post_prune(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), testing_D, A, parent=current, current=node)
        
        tmp_node=Node(feature_name='leaf-'+str(self.leaf_count), isLeaf=True, classification=most_frequent)
        
        if parent:
            # when current node is not the root
            for key, node in parent.map.items():
                if node==current:
                    parent.map[key]=tmp_node
                    saved_key=key
                    break
            # compare the evaluation, if it is enhanced then prune the tree.
            tmp_accuracy=self.evaluate(testing_D, A)
            if tmp_accuracy<self.current_accuracy:
                parent.map[saved_key]=current
            else:
                self.current_accuracy=tmp_accuracy
                self.leaf_count+=1
            return
        else:
            # when current node is the root
            saved_tree=self.tree
            self.tree=tmp_node
            tmp_accuracy=self.evaluate(testing_D, A)
            if tmp_accuracy<self.current_accuracy:
                self.tree=saved_tree
            else:
                self.current_accuracy=tmp_accuracy
                self.leaf_count+=1
            return

    def predict(self, D, A):
        '''
        Predict the classification for the data in D.
        For the definition of A, see method 'fit'. 
        There is one critical difference between D and that defined in 'fit':
            the last column may or may not be the labels. 
            This method works as long as the feature index in A matches the corresponding
            column in D.
        '''
        row, _=D.shape
        pred=np.empty((row, 1), dtype=str)
        tmp_data={key: None for key in A.keys()}
        for i in range(len(D)):
            for key, info in A.items():
                tmp_data[key]=D[i, info[0]]
            pred[i]=self.tree(tmp_data)
        return pred
    
    def evaluate(self, testing_D, A):
        '''
        Evaluate the performance of decision tree. (Accuracy)
        For definition of testing_D and A, see 'predict'.
        However, here the testing_D is required to be labelled, that is, its last column 
        should be labels of the data.
        '''
        true_label=testing_D[:, -1]
        pred_label=self.predict(testing_D, A)
        
        success_count=0
        for i in range(len(true_label)):
            if true_label[i]==pred_label[i]:
                success_count+=1

        return success_count/len(true_label)


The tree "works" with Adult Data. 

In [109]:

dataset = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
row_names = ["Age", "Workclass", "Fnlwgt", "Education", "EducationNum", "MaritalStatus",
        "Occupation", "Relationship", "Race", "Gender", "CapitalGain", "CapitalLoss",
        "HoursPerWeek", "Country", "Income"]
us_adult_income = pd.read_csv(dataset, names=row_names,na_values=[' ?'])
us_adult_income["Income"] = pd.Categorical(us_adult_income["Income"])
us_adult_income["Income"] = us_adult_income["Income"].cat.codes
us_adult_income["Income"] = 1 - us_adult_income["Income"]

us_adult_income.head()

,Age,Workclass,Fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,Country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1


In [110]:
adult2 = us_adult_income.sample(frac=0.01, random_state=42)
adult2.shape

(326, 15)

In [157]:
adult25 = adult2.copy()
adult25["Workclass"] = pd.Categorical(adult25["Workclass"])
adult25["Workclass"] = adult25["Workclass"].cat.codes

adult25["Education"] = pd.Categorical(adult25["Education"])
adult25["Education"] = adult25["Education"].cat.codes

adult25["MaritalStatus"] = pd.Categorical(adult25["MaritalStatus"])
adult25["MaritalStatus"] = adult25["MaritalStatus"].cat.codes

adult25["Occupation"] = pd.Categorical(adult25["Occupation"])
adult25["Occupation"] = adult25["Occupation"].cat.codes

adult25["Relationship"] = pd.Categorical(adult25["Relationship"])
adult25["Relationship"] = adult25["Relationship"].cat.codes

adult25["Race"] = pd.Categorical(adult25["Race"])
adult25["Race"] = adult25["Race"].cat.codes

adult25["Gender"] = pd.Categorical(adult25["Gender"])
adult25["Gender"] = adult25["Gender"].cat.codes

adult25["Country"] = pd.Categorical(adult25["Country"])
adult25["Country"] = adult25["Country"].cat.codes

In [197]:
adult23 = adult25[["Age", "Education", "Occupation", "Gender", "CapitalLoss", "Income"]]
adult23

,Age,Education,Occupation,Gender,CapitalLoss,Income
14160,27,13,0,0,0,1
27048,45,10,2,0,0,1
28868,29,8,2,1,0,0
5667,30,8,5,0,0,1
7827,29,13,1,1,0,1
...,...,...,...,...,...,...
18780,26,8,8,0,0,1
14969,59,1,1,1,0,1
3383,40,7,1,1,0,1
15301,39,13,12,1,0,1


In [104]:
conf

{'A': {'sepal length': [0, 7.05],
  'sepal width': [1, 2.55],
  'petal length': [2, 5.35],
  'petal width': [3, 1.55]},
 'feature_discrete': {'sepal length': False,
  'sepal width': False,
  'petal length': False,
  'petal width': False},
 'treeType': 'IGGR'}

In [ ]:
D= np.genfromtxt('datafiles/iris_train.txt', dtype=str)
np.random.shuffle(D)

D

In [176]:
D4 = D2.astype(str)
D4

array([['62', '1', '136787', ..., '40', '13', '1'],
       ['66', '2', '180211', ..., '30', '10', '1'],
       ['45', '2', '38950', ..., '40', '13', '1'],
       ...,
       ['46', '3', '168796', ..., '55', '13', '1'],
       ['40', '2', '289309', ..., '48', '13', '1'],
       ['29', '4', '189346', ..., '50', '13', '1']], dtype='<U21')

In [203]:
D2

array([['37', '9', '8', '0', '2559', '0'],
       ['37', '10', '4', '1', '0', '1'],
       ['90', '6', '0', '1', '0', '1'],
       ...,
       ['54', '10', '0', '1', '0', '1'],
       ['24', '10', '2', '1', '0', '1'],
       ['46', '10', '1', '1', '0', '0']], dtype='<U21')

In [204]:
len(D2)

326

In [ ]:
#D2= np.genfromtxt('datafiles/iris_train.txt', dtype=str)
D2 = adult23.to_numpy()
D2 = D2.astype(str)
np.random.shuffle(D2)

k=len(D2)

# attribute_name=['sepal length', 'sepal width', 'petal length', 'petal width']

attribute_name = ["Age", "Education", "Occupation", "Gender", "CapitalLoss"]

A={name: [i] for i, name in enumerate(attribute_name)}
# feature_discrete = {name: False for name in attribute_name}
feature_discrete={name: False if isinstance(adult2[name].iloc[0], np.integer) else True for name in attribute_name}

conf={'A': A, 
      'feature_discrete': feature_discrete, 
      'treeType':'IGGR'}

dt=decisionTree(conf)
dt.train(D2[:len(D2)//2])
print(dt.tree)
print(dt.eval(D2[len(D2)//2:]))
dt.prune(D2[:len(D2)//2], D2[len(D2)//2:])
print(dt.tree)
print(dt.eval(D2[len(D2)//2:]))


In [237]:
len(D2[len(D2)//2:])

163

In [199]:
feature_discrete # both are in use now

{'Age': False,
 'Education': True,
 'Occupation': True,
 'Relationship': True,
 'Gender': True,
 'CapitalLoss': False}

#### 0.94 0.96, 0.98 1.0 - IGGR

#### 0.92 0.94 0.96 0.98 - C4.5

In [33]:
conf

{'A': {'sepal length': [0],
  'sepal width': [1],
  'petal length': [2],
  'petal width': [3]},
 'feature_discrete': {'sepal length': False,
  'sepal width': False,
  'petal length': False,
  'petal width': False},
 'treeType': 'CART'}

In [39]:
print(dt.tree)

============Root===========
[petal width]: {<= 0.75 leaf-0, > 0.75 petal length}

[leaf-0]: {0} +++ [petal length]: {<= 4.75 leaf-1, > 4.75 petal width}

[leaf-1]: {1} +++ [petal width]: {<= 1.55 petal width, > 1.55 leaf-4}

[petal width]: {<= 1.45 leaf-2, > 1.45 leaf-3} +++ [leaf-4]: {2}

[leaf-2]: {2} +++ [leaf-3]: {1}
============Leaf===========


Decision Tree 

Code (modified) from https://github.com/sam-fletcher/SNR_DP_Forest

In [249]:
from collections import Counter, defaultdict
import random
import numpy as np
import math
import pandas as pd 
import numpy as np 

In [254]:
# currently includes methods as:
        # _init_  : _trees
        # get_domains
        # reduce trees
        # evaluate_accuracy_with_voting

class DPRF_Forest:  
    def __init__(self, 
                 training_data, # 2D list of the training data where the columns are the attributes, and the first column is the class attribute
                 # test_data, # T
                 epsilon, # epsilon, Budget, B, the total privacy budget
                 f, # n of attributes to be split used by each dctree f
                 max_depth # max tree depth 
                 ):

        self._trees = []


        ''' Some initialization '''
        # Attribute set F
        attribute_values = self.get_domains(training_data)

        # Class values set
        class_values = [str(y) for y in list(set([x[len(x)-1] for x in training_data]))]
        attribute_indexes = [int(k) for k,v in attribute_values.items()]


        print(attribute_values)
        print("------------")
        print(class_values)
        print("------------")
        print(attribute_indexes)
        print("-----------")

        
        # |D|
        dataset_size = len(training_data)


        valid_attribute_sizes = [[int(k),len(v)] for k,v in attribute_values.items()]

        print("valid attribute sizes")
        print(valid_attribute_sizes)

        # Number of trees t
        num_trees = len(attribute_indexes)

        average_domain = np.mean( [x[1] for x in valid_attribute_sizes] )

        # e = B / t
        epsilon_per_tree = epsilon / float(num_trees)

        ''' Calculating minimum support threshold '''
        # estimated_support_min_depth = dataset_size / (average_domain**2) # a large number
        # estimated_support_max_depth = dataset_size / (average_domain ** (len(attribute_indexes)/2)) # max tree depth is k/2 # a small number
        # epsilon_per_tree = epsilon / float(num_trees)
        # min_support = len(class_values) * math.sqrt(2) * (1/epsilon_per_tree) * SNR_MULTIPLIER # the minimum support in order for S>N


        # while estimated_support_max_depth > min_support: # then we can have more trees
        #    num_trees += 1
        #    epsilon_per_tree = epsilon / float(num_trees)
        #    min_support = len(class_values) * math.sqrt(2) * (1/epsilon_per_tree) * SNR_MULTIPLIER


        # while min_support > estimated_support_min_depth and num_trees>1: # then we need to have less trees
        #    num_trees, valid_attribute_sizes, estimated_support_min_depth = self.reduce_trees(num_trees, valid_attribute_sizes, dataset_size)
        #    epsilon_per_tree = epsilon / float(num_trees)
        #    min_support = len(class_values) * math.sqrt(2) * (1/epsilon_per_tree) * SNR_MULTIPLIER


        print("NUM TREES = {} & EPSILON PER TREE = {}".format(num_trees, epsilon_per_tree))

        # ? 
        root_attributes = []

       
        for a in attribute_indexes:
            if a in [x[0] for x in valid_attribute_sizes]:
                root_attributes.append(a) # OR: [index, support, gini]

        print("root attributes")
        print(root_attributes)

        # for treeId = 1,2 ... t do:
        for t in range(num_trees):

            if not root_attributes:
                root_attributes = attribute_indexes[:]

            # randomly extract |D| samples from D w/ a bagging algo?
            root = random.choice(root_attributes)
            root_attributes.remove(root)

            # TREE BUILDTREE(D, A, C, eps, f, Dm)
            # D :
            tree = Tree_DPDT(attribute_indexes, attribute_values, root, class_values, dataset_size, epsilon_per_tree) 
           
            # TO EVOLVE HERE 

            # attribute_name = ["Age", "Education", "Occupation", "Gender", "CapitalLoss"]

            # A={name: [i] for i, name in enumerate(attribute_name)}
            # feature_discrete = {name: False for name in attribute_name}
            # feature_discrete={name: False if isinstance(adult2[name].iloc[0], np.integer) else True for name in attribute_name}

            # conf={'A': A, 
            #      'feature_discrete': feature_discrete, 
            #       'treeType':'IGGR'}
            # tree = decisionTree(conf)

            num_unclassified = tree.filter_training_data_and_count(training_data, epsilon_per_tree, class_values)
            
            if num_unclassified > 0:
                print("number of unclassified records = {}".format(num_unclassified))
            tree.prune_tree()

            # FOREST = FOREST U TREE
            self._trees.append(tree)


    def get_domains(self, data):
        attr_domains = {}
        transData = np.transpose(data)
        for i in range(0,len(data[0]) - 1):
            attr_domains[str(i)] = [str(x) for x in set(transData[i])]
            print("original domain length of categorical attribute {}: {}".format(i, len(attr_domains[str(i)])))
        return attr_domains


    def reduce_trees(self, num_trees, valid_attribute_sizes, dataset_size):
        num_trees -= 1
        largest_attribute = sorted(valid_attribute_sizes, key=lambda x:x[1], reverse=True)[0]
        #print("Removing att{} with domain size {}".format(largest_attribute[0], largest_attribute[1])) 
        new_valids = [ x for x in valid_attribute_sizes if x[0] != largest_attribute[0] ]
        average_domain = np.mean( [x[1] for x in valid_attribute_sizes] ) # average with all attributes
        narrowed_average_domain = np.mean([x[1] for x in valid_attribute_sizes if x[0] in [y[0] for y in new_valids] ]) # average without the big attributes
        estimated_support_squared = dataset_size / (average_domain * narrowed_average_domain)
        return num_trees, new_valids, estimated_support_squared


    # GET MAJORITY LABELS (Forest, T, C) equivalent from pseudocode
    def evaluate_accuracy_with_voting(self, records, class_index=0):

        ''' Calculate the Prediction Accuracy of the Forest. '''
        actual_labels = [x[class_index] for x in records]
        predicted_labels = []
        leafs_not_used = 0
        count_of_averages_used = 0

        # FOR EACH RECORD X IN DATASET DO
        for rec in records:

            class_value_fractions = defaultdict(list)

            # FOR EACH TREE DO
            for tree in self._trees:

                # GET PREDICTED CLASSIFICATION RESULT
                node, leaf_not_used = tree._classify(tree._root_node, rec)

                noisy_class_counts = node._noisy_class_counts
                leafs_not_used += leaf_not_used

                support = float(sum([v for k,v in noisy_class_counts.items()]))
                for k,v in noisy_class_counts.items():
                    class_value_fractions[k].append(v/support)
            best_confidences = {}

            for k,lis in class_value_fractions.items():
               best_confidences[k] = max(lis)
            best = [None, 0.0]

            for k,class_best in best_confidences.items():
                if class_best > best[1]:
                    best = [k, class_best]
            average_used = False

            for k,class_best in best_confidences.items():

                if class_best == best[1] and k != best[0]:

                    average_used = True
                    #print("original best: {} vs. contender: {}".format(best, (k,class_best)))
                    orig_average = np.mean(class_value_fractions[best[0]])
                    contender_average = np.mean(class_value_fractions[k])
                    #print("original average: {} vs. contender: {}".format(orig_average, contender_average))
                    if contender_average > orig_average:
                        best = [k, class_best]

            count_of_averages_used += 1 if average_used else 0
            predicted_labels.append(int(best[0]))

        counts = Counter([x == y for x, y in zip(predicted_labels, actual_labels)])
        
        # acc = float(counts[True]) / len(records)
        
        return float(counts[True]) / len(records),   leafs_not_used / (len(records)*len(self._trees)),   count_of_averages_used / len(records)



In [255]:

# Input:
    # Training set D 
    # Privacy budget B (epsilon)

    # Test Set T (?)  (Maybe?)

    # Number of attributes to be split used by each dec tree f 
    # Max tree depth d_m

B = 0.2
f = 3
dm = 5

diff_priv_forest = DPRF_Forest(D2[100:], B, f, dm)

original domain length of categorical attribute 0: 55
original domain length of categorical attribute 1: 13
original domain length of categorical attribute 2: 13
original domain length of categorical attribute 3: 2
original domain length of categorical attribute 4: 10
{'0': ['39', '51', '49', '22', '31', '25', '66', '34', '43', '28', '52', '48', '54', '53', '27', '42', '64', '35', '26', '59', '36', '81', '18', '37', '33', '20', '57', '50', '67', '61', '58', '62', '29', '24', '45', '19', '69', '30', '72', '63', '38', '41', '17', '56', '40', '46', '21', '32', '90', '60', '55', '68', '47', '23', '44'], '1': ['5', '0', '8', '6', '1', '10', '9', '2', '4', '7', '11', '13', '12'], '2': ['3', '5', '0', '8', '6', '1', '10', '2', '9', '4', '-1', '11', '12'], '3': ['0', '1'], '4': ['0', '1617', '1887', '2377', '2559', '1902', '1876', '2051', '1977', '1848']}
------------
['0', '1']
------------
[0, 1, 2, 3, 4]
-----------
valid attribute sizes
[[0, 55], [1, 13], [2, 13], [3, 2], [4, 10]]
NUM TREE

NameError: name 'Tree_DPDT' is not defined

In [240]:
# Here the classification result set is the left column 
D2[100:]

array([['42', '10', '1', '0', '0', '1'],
       ['50', '10', '1', '1', '0', '0'],
       ['18', '1', '6', '0', '0', '1'],
       ...,
       ['47', '10', '0', '0', '0', '1'],
       ['32', '8', '8', '1', '0', '1'],
       ['29', '13', '0', '1', '0', '1']], dtype='<U21')